In [1]:
%load_ext autoreload
%autoreload 2

from sae_lens import SAE, ActivationsStore
from transformer_lens import HookedTransformer

In [2]:
def load_model_and_sae(
    model_name: str, sae_release: str, sae_id: str, device: str
) -> tuple:
    model = HookedTransformer.from_pretrained(model_name, device=device)
    sae, _, _ = SAE.from_pretrained(release=sae_release, sae_id=sae_id, device=device)
    sae.W_dec.norm(dim=-1).mean()
    sae.fold_W_dec_norm()
    return model, sae

In [3]:
model_gemma, sae_gemma = load_model_and_sae(
    "gemma-2-2b",
    "gemma-scope-2b-pt-res-canonical",
    "layer_18/width_16k/canonical",
    "mps",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


In [4]:
model_gpt2, sae_gpt2 = load_model_and_sae(
    "gpt2-small",
    "gpt2-small-res-jb-feature-splitting",
    "blocks.8.hook_resid_pre_768",
    "mps",
)

Loaded pretrained model gpt2-small into HookedTransformer


/Users/matthew/Library/Caches/pypoetry/virtualenvs/sae-cooccurence-DZTJ6ajw-py3.11/lib/python3.11/site-packages/sae_lens/sae.py:136: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


In [5]:
activation_store_gpt2 = ActivationsStore.from_sae(
    model=model_gpt2,
    sae=sae_gpt2,
    streaming=True,
    store_batch_size_prompts=8,
    train_batch_size_tokens=512,
    n_batches_in_buffer=16,
    device="mps",
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/matthew/Library/Caches/pypoetry/virtualenvs/sae-cooccurence-DZTJ6ajw-py3.11/lib/python3.11/site-packages/sae_lens/training/activations_store.py:245: UserWarning: Dataset is not tokenized. Pre-tokenizing will improve performance and allows for more control over special tokens. See https://jbloomaus.github.io/SAELens/training_saes/#pretokenizing-datasets for more info.
  warnings.warn(


In [6]:
# activation_store_gemma = ActivationsStore.from_sae(
#     model=model_gemma,
#     sae=sae_gemma,
#     streaming=True,
#     store_batch_size_prompts=8,
#     train_batch_size_tokens=512,
#     n_batches_in_buffer=16,
#     device="mps",
# )

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
activations_batch_old = activation_store_gpt2.next_batch()
activations_batch_old.shape

Token indices sequence length is longer than the specified maximum sequence length for this model (1217 > 1024). Running this sequence through the model will result in indexing errors


torch.Size([512, 1, 768])

In [25]:
sae_gpt2.encode(activations_batch_old).shape

torch.Size([512, 1, 768])

In [9]:
batch_size = activation_store_gpt2.train_batch_size_tokens
batch_size

512

In [14]:
batch_tokens = activation_store_gpt2.get_batch_tokens(batch_size)
# batch_tokens = batch_tokens[:, 1:, ...]
batch_tokens

tensor([[50256,   257,  1256,  ...,   465,    11,   326],
        [50256,   447,   247,  ...,  1406,  5543,   356],
        [50256,  1410,   284,  ...,    72,  4893,   262],
        ...,
        [50256,   262,  8425,  ...,   850,    12, 35448],
        [50256,  4249,  7310,  ..., 16686,   284,   257],
        [50256,  2060, 15815,  ...,   517,  3665,  4899]], device='mps:0')

In [23]:
batch_tokens.shape

torch.Size([512, 128])

In [16]:
activations = activation_store_gpt2.get_activations(batch_tokens)
activations.shape

torch.Size([512, 128, 1, 768])

In [17]:
activations_wout_bos = activations[:, 1:, ...]
activations_wout_bos.shape

torch.Size([512, 127, 1, 768])

In [22]:
# flattened_activations = activations_wout_bos.view(-1, activation_store_gpt2.d_in)
flattened_activations = activations_wout_bos.reshape(-1, 1, activation_store_gpt2.d_in)
flattened_activations.shape

torch.Size([65024, 1, 768])